In [177]:
import cv2 as cv 
import numpy as np
import os
import matplotlib.pyplot as plt
import copy

In [178]:
class Line:
    """
    Store a line based on the two points.
    """
    def __init__(self, point_1, point_2):
        self.point_1 = point_1
        self.point_2 = point_2
        

class Point:
    def __init__(self, x, y):
        self.x = np.int32(np.round(x))
        self.y = np.int32(np.round(y))
    
    def get_point_as_tuple(self):
        return (self.x, self.y)
    
        
class Patch:
    """
    Store information about each item (where the mark should be found) in the table. 
    """
    def __init__(self, image_patch, x_min, y_min, x_max, y_max, line_idx, column_idx):
        self.image_patch = image_patch
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        self.line_idx = line_idx
        self.column_idx = column_idx
        self.domino_value = -1 #no domino
    
    def set_domino_value(self, domino_value: int):
        assert any([domino_value == i-1 for i in range(8)])
        self.domino_value = domino_value

In [179]:
def show_image(image, window_name='image', timeout=0):
    """
    :param timeout. How many seconds to wait untill it close the window.
    """
    cv.imshow(window_name, cv.resize(image, None, fx=0.6, fy=0.6))
    cv.waitKey(timeout)
    cv.destroyAllWindows()

def draw_lines(image, lines: list[Line], timeout: int = 0, color: tuple = (0, 0, 255),
               return_drawing: bool = False, window_name: str = 'window'):
    """
    Plots the lines into the image.
    :param image.
    :param lines.
    :param timeout. How many seconds to wait untill it close the window.
    :param color. The color used to draw the lines
    :param return_drawing. Use it if you want the drawing to be return instead of displayed.
    :return None if return_drawing is False, otherwise returns the drawing.
    """
    drawing = image.copy()
    if drawing.ndim == 2:
        drawing = cv.cvtColor(drawing, cv.COLOR_GRAY2BGR)
    for line in lines: 
        cv.line(drawing, line.point_1.get_point_as_tuple(), line.point_2.get_point_as_tuple(), color, 2, cv.LINE_AA)
        
    if return_drawing:
        return drawing
    else:
        show_image(drawing, window_name=window_name, timeout=timeout)

In [180]:
def get_patches(lines: list[Line], columns: list[Line], image, show_patches: bool = False) -> list[Patch]:
    """
    It cuts out each box from the table defined by the lines and columns.
    :param lines. The lines that difine the table.
    :param columns. The columns that difine the table.
    :param image. The image containing the table.
    :param show_patches. Determine if the patches will be drawn on the image or not.
    :return : A list with all boxes in the table.
    """
    
    def crop_patch(image_, x_min, y_min, x_max, y_max):
        """
        Crops the bounding box represented by the coordinates.
        """
        return image_[y_min: y_max, x_min: x_max].copy()
    
    def draw_patch(image_, patch: Patch, color: tuple = (255, 0, 255)):
        """
        Draw the bounding box corresponding to the patch on the image.
        """
        cv.rectangle(image_, (patch.x_min, patch.y_min), (patch.x_max, patch.y_max), color=color, thickness=5)
    
    assert image.ndim == 2
    if show_patches: 
        image_color = np.dstack((image, image, image))
  
    lines.sort(key=lambda line: line.point_1.y)
    columns.sort(key=lambda column: column.point_1.x)
    patches = []
    step = 0
    for line_idx in range(len(lines) - 1):
        for col_idx in range(len(columns) - 1):
            current_line = lines[line_idx]
            next_line = lines[line_idx + 1] 
            
            y_min = current_line.point_1.y + step
            y_max = next_line.point_1.y - step
            
            current_col = columns[col_idx]
            next_col = columns[col_idx + 1]
            x_min = current_col.point_1.x + step 
            x_max = next_col.point_1.x - step
            
            patch = Patch(image_patch=crop_patch(image,  x_min, y_min, x_max, y_max),
                          x_min=x_min, y_min=y_min, x_max=x_max, y_max=y_max,
                          line_idx=line_idx, column_idx=col_idx)
            
            if show_patches:
                draw_patch(image_color, patch)
            
            patches.append(patch)
            
    if show_patches:
        show_image(image_color, window_name='patches', timeout=0)
    return patches

In [181]:
def find_patches_template(image, template, show_intermediate_results: bool = False):
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    query_image_edges = cv.Canny(image_gray, 200, 250) #the params were found by trial and error
    template_image_edges = cv.Canny(template_gray, 200, 250)
    
    if show_intermediate_results: 
        show_image(query_image_edges, window_name='query_image_edges', timeout=0)
        show_image(template_image_edges, window_name='template_image_edges', timeout=0)
        
    alligned_image = image
    
    left_corner = Point(x=0, y=0)
    right_corner = Point(x=len(template), y=len(template))

    step_x = (right_corner.x - left_corner.x) / 15
    step_y = (right_corner.y - left_corner.y) / 15
    
    def get_vertical_and_horizontal_lines(left_corner: Point, step_x_, step_y_): 
        vertical_lines = []
        horizontal_lines = []
        
        for i in range(16):
            xmin = np.uint64(left_corner.x + i * step_x_)
            line = Line(Point(x = xmin, y = 0), Point(x = xmin, y = 5000)) 
            vertical_lines.append(line)
        for i in range(16):
            ymin = np.uint64(left_corner.y + i * step_y_)
            line = Line(Point(x = 0, y = ymin), Point(x = 5000, y = ymin))
            horizontal_lines.append(line)

        return vertical_lines, horizontal_lines
     
    vertical_lines, horizontal_lines = get_vertical_and_horizontal_lines(left_corner, step_x, step_y) 
    
    alligned_image_gray = cv.cvtColor(alligned_image, cv.COLOR_BGR2GRAY)
    patches = get_patches(horizontal_lines, vertical_lines, alligned_image_gray, 
                          show_patches=show_intermediate_results)
    
    return patches

In [182]:
class DominoClassifier:
    """
    Classifier that determines the value of the domino
    """
    def __init__(self):
        self.whites = [0 for _ in range(6)]
        self.blacks = [0 for _ in range(6)]
    
    def classify(self, patch: Patch) -> int:
        """
        Uses Hough circles to determine the domino value
        """
        def is_domino(patch):
            """
            Receive a (greyscale) Patch and return True if it is classified as a domino and False otherwise
            """
            _, thresh = cv.threshold(patch, 185, 255, cv.THRESH_BINARY) #thresh value found through trial and error

            sum_whites = np.sum(thresh == 255)
            sum_blacks = np.sum(thresh == 0)

            if sum_whites >= min(self.whites) and sum_whites <= max(self.whites) and sum_blacks >= min(self.blacks) and sum_blacks <= max(self.blacks):
                return True
            
            return False

        if not is_domino(patch):
            return -1

        circles = cv.HoughCircles(patch, cv.HOUGH_GRADIENT, 1, patch.shape[0] / 8, param1=100, param2=30, minRadius=5, maxRadius=15)
        if circles is not None:
            circles = np.uint16(np.around(circles))
            # cv.imshow('patch', patch)
            # cv.waitKey(0)
            # cv.destroyAllWindows()
            # for i in circles[0,:]:
            #     # draw the outer circle
            #     cv.circle(patch_copy,(i[0],i[1]),i[2],(0,255,0),2)
            #     # draw the center of the circle
            #     cv.circle(patch_copy,(i[0],i[1]),2,(0,0,255),3)

            # cv.imshow('found circles', patch_copy)
            # cv.waitKey(0)
            # cv.destroyAllWindows()
            return len(circles[0,:]) # number of circles
        
        return -1 # for false positives
        
    def get_domino_whites_and_blacks(self, dominoes_path='dominoes/'):   
        domino_filenames = [dominoes_path + f for f in os.listdir(dominoes_path) if f.endswith('.jpg')]
        whites = []
        blacks = []

        for domino_filename in domino_filenames:
            domino = cv.imread(domino_filename)
            domino = cv.cvtColor(domino, cv.COLOR_BGR2GRAY)

            _, thresh = cv.threshold(domino, 185, 255, cv.THRESH_BINARY)

            whites.append(np.sum(thresh == 255)) 
            blacks.append(np.sum(thresh == 0))

        self.whites = whites
        self.blacks = blacks
        return whites, blacks

def classify_patches_with_domino_classifier(patches: list[Patch], dominoes_path: str = 'dominoes/') -> None:
    """
    Receive the patches and classify if the patch contains an 'X' or not.
    :param patches.
    :return None
    """
    domino_classifier = DominoClassifier()
    domino_classifier.get_domino_whites_and_blacks(dominoes_path)
    for line in patches:
        for column in line:
            column.set_domino_value(domino_classifier.classify(column.image_patch))

In [204]:
score_to_domino_dict = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 0,
    8: 2,
    9: 5,
    10: 3,
    11: 4,
    12: 6,
    13: 2,
    14: 2,
    15: 0,
    16: 3,
    17: 5,
    18: 4,
    19: 1,
    20: 6,
    21: 2,
    22: 4,
    23: 5,
    24: 5,
    25: 0,
    26: 6,
    27: 3,
    28: 4,
    29: 2,
    30: 0,
    31: 1,
    32: 5,
    33: 1,
    34: 3,
    35: 4,
    36: 4,
    37: 4,
    38: 5,
    39: 0,
    40: 6,
    41: 3,
    42: 5,
    43: 4,
    44: 1,
    45: 3,
    46: 2,
    47: 0,
    48: 0,
    49: 1,
    50: 1,
    51: 2,
    52: 3,
    53: 6,
    54: 3,
    55: 5,
    56: 2,
    57: 1,
    58: 0,
    59: 6,
    60: 6,
    61: 5,
    62: 2,
    63: 1,
    64: 2,
    65: 5,
    66: 0,
    67: 3,
    68: 3,
    69: 5,
    70: 0,
    71: 6,
    72: 1,
    73: 4,
    74: 0,
    75: 6,
    76: 3,
    77: 5,
    78: 1,
    79: 4,
    80: 2,
    81: 6,
    82: 2,
    83: 3,
    84: 1,
    85: 6,
    86: 5,
    87: 6,
    88: 2,
    89: 0,
    90: 4,
    91: 0,
    92: 1,
    93: 6,
    94: 4,
    95: 4,
    96: 1,
    97: 6,
    98: 6,
    99: 3,
    100: 0
}

positions_to_points = {
    (1,1): 5,
    (1,4): 4,
    (1,8): 3,
    (1,12): 4,
    (15,15): 5,
    (2, 3): 3,
    (2, 6): 4,
    (2, 10): 4,
    (2, 13): 3,
    (3, 2): 3,
    (3, 5): 2,
    (3, 11): 2,
    (3, 14): 3,
    (4, 1): 4,
    (4, 4): 3,
    (4, 6): 2,
    (4, 10): 2,
    (4, 12): 3,
    (4, 15): 4,
    (5, 3): 2,
    (5, 5): 1,
    (5, 7): 1,
    (5, 9): 1,
    (5, 11): 1,
    (5, 13): 2,
    (6, 2): 4,
    (6, 4): 2,
    (6, 6): 1,
    (6, 10): 1,
    (6, 12): 2,
    (6, 14): 4,
    (7, 5): 1,
    (7, 11): 1,
    (8, 1): 3,
    (8, 15): 3,
    (9, 5): 1,
    (9, 11): 1,
    (10, 2): 4,
    (10, 4): 2,
    (10, 6): 1,
    (10, 10): 1,
    (10, 12): 2,
    (10, 14): 4,
    (11, 3): 2,
    (11, 5): 1,
    (11, 7): 1,
    (11, 9): 1,
    (11, 11): 1,
    (11, 13): 2,
    (12, 1): 4,
    (12, 4): 3,
    (12, 6): 2,  
    (12, 10): 2,
    (12, 12): 3,
    (12, 15): 4,
    (13, 2): 3,
    (13, 5): 2,
    (13, 11): 2,
    (13, 14): 3,
    (14, 3): 3,
    (14, 6): 4,
    (14, 10): 4,
    (14, 13): 3,
    (15, 1): 5,
    (15, 4): 4,
    (15, 8): 3,
    (15, 12): 4,
    (15, 15): 5
}


In [184]:
def get_game_move_to_patches(images_path = 'aligned_train/'):
    template = cv.imread(images_path + 'aligned_1_01.jpg')
    game_to_patches = {}

    img_filenames = [images_path + f for f in os.listdir(images_path) if f.endswith('.jpg') and f.startswith('aligned')]
    for img_filename in img_filenames:
        original_image = cv.imread(img_filename) 
        patches = find_patches_template(original_image, template)
        patches = np.reshape(patches, (15,15))
        game, move = img_filename.split('/')[-1].split('.')[0].split('_')[1:3]
        game_to_patches[(game, move)] = patches
    return game_to_patches

In [208]:
def whose_move(game, moves_path = 'train/regular_tasks/'):
    player_to_moves_filename = moves_path + game + '_moves.txt' 

    with open(player_to_moves_filename, 'r') as f:
        lines = f.readlines()

    move_to_player_dict = {}
    for line in lines:
        if line != '' and line != '\n':
            line_split = line.split(' ')
            move = line_split[0].split('.')[0].split('_')[-1]
            if line_split[-1] == 'player1':
                move_to_player_dict[move] = 'player1'
            else:
                move_to_player_dict[move] = 'player2'
    
    return move_to_player_dict


def get_scores(images_path = 'aligned_train/', out_path = 'scores/', moves_path = 'train/regular_tasks/'):
    if not os.path.exists(out_path):
        os.mkdir(out_path)
        os.mkdir(out_path + 'regular_tasks/')
    out_path = out_path + 'regular_tasks/'

    game_to_patches = get_game_move_to_patches(images_path)

    games_to_moves = {} # better data structure to store data
    for item in game_to_patches:
        if item[0] not in games_to_moves:
            games_to_moves[item[0]] = []
        games_to_moves[item[0]].append({item[1]: game_to_patches[(item[0], item[1])]})

    for game in games_to_moves.keys():
        game_to_player_moves = whose_move(game, moves_path)
        player1_position = 0
        player2_position = 0

        domino_list = []
        for move_to_patches in games_to_moves[game]:
            individual_move = [item for item in move_to_patches.items()][0]
            move = individual_move[0]

            player = game_to_player_moves[move]
            player1_move_score = 0
            player2_move_score = 0

            classify_patches_with_domino_classifier(game_to_patches[(game, move)])
            with open(out_path + game + '_' + move + '.txt', 'w') as f:
                for i, line in enumerate(game_to_patches[(game, move)]):
                    for j, column in enumerate(line):
                        if column.domino_value != -1:
                            position = str(i + 1) + str(chr(ord('@') + j + 1))
                            value = column.domino_value

                            if position not in domino_list:
                                output = position + ' ' + str(value) 
                                domino_list.append(position)
                                pos_tuple = (i+1,j+1)
                                if pos_tuple in positions_to_points.keys() and player == 'player1':
                                    if player1_position != 0 and score_to_domino_dict[player1_position] == value: 
                                        player1_move_score += positions_to_points[pos_tuple] + 3
                                    else:
                                        player1_move_score += positions_to_points[pos_tuple]
                                    player1_position += player1_move_score
                                elif pos_tuple in positions_to_points.keys() and player == 'player2': 
                                    if player2_position != 0 and score_to_domino_dict[player2_position] == value:
                                        player2_move_score += positions_to_points[pos_tuple] + 3
                                    else:
                                        player2_move_score += positions_to_points[pos_tuple]
                                    player2_position += player2_move_score

                                f.write(output + '\n')
                if player == 'player1':
                    f.write(str(player1_move_score) + '\n')
                elif player == 'player2':
                    f.write(str(player2_move_score) + '\n')

images_path = 'aligned_train/'
out_path = 'scores/'
get_scores(images_path, out_path)